In [1]:
#build some directories required
!mkdir exported
!mkdir exported/gpt2_model/

mkdir: exported: File exists
mkdir: exported/gpt2_model/: File exists


In [2]:
# https://huggingface.co/learn/nlp-course/en/chapter7/6
# data loading and prep
# load dataset from a file and split into train and test

from datasets import load_dataset

train_ds, test_ds = load_dataset(    
    "text", # type of data
    data_files={
        "this": ["dummy_data/5000_oscar.eo.txt"], # provide keyword helps split data later
    },
    split=["this[:80%]", "this[-20%:]"] # first 80% is train; last 20% test
)

/Users/saideeptiku/.miniconda3/envs/hf/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# https://github.com/huggingface/notebooks/blob/main/examples/tokenizer_training.ipynb
# train a new tokenizer
from transformers import AutoTokenizer, LlamaTokenizerFast

# function/generator to produce batches of data
from dataclasses import dataclass # i like to give type hints
def batch_iterator(dataset: dataclass, batch_size: int, dtype="text"):
    # iter over each row of the dataset
    # imagine a file with many sentences
    # and produce a sub-sentence of a predefined length
    for i in range(0, len(dataset), batch_size):
        yield dataset[i : i + batch_size][dtype]

# train a new tokenizer using the previous one with 
tokenizer = AutoTokenizer.from_pretrained("gpt2").train_new_from_iterator(
    batch_iterator(train_ds, 10), 
    vocab_size=25000
    )

# save the tokenizer files
# also saves vocab; yes idk what that actually is 
tokenizer.save_pretrained("exported/gpt2_model/tokenizer/")

/Users/saideeptiku/.miniconda3/envs/hf/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


('exported/gpt2_model/tokenizer/tokenizer_config.json',
 'exported/gpt2_model/tokenizer/special_tokens_map.json',
 'exported/gpt2_model/tokenizer/vocab.json',
 'exported/gpt2_model/tokenizer/merges.txt',
 'exported/gpt2_model/tokenizer/added_tokens.json',
 'exported/gpt2_model/tokenizer/tokenizer.json')

In [4]:
# this needs to be fixed 
# context length
CTX_LEN = 32

# time to build the model
# https://huggingface.co/learn/nlp-course/en/chapter7/6
from transformers import GPT2LMHeadModel, AutoConfig

config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=CTX_LEN,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

model = GPT2LMHeadModel(config)
model_size = sum(t.numel() for t in model.parameters())
print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")

GPT-2 size: 87.2M parameters


In [5]:
from transformers import DataCollatorForLanguageModeling
# set padding token same as end of sentence token
tokenizer.pad_token = tokenizer.eos_token
# data collator helps add padding to sentences with length longer than ctx_len
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [6]:
def tokenize(element: dataclass, context_length: int = CTX_LEN, dtype="text"):
    outputs = tokenizer(
        element[dtype],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    # note that we asked for length to be returned
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}

# tokenize using map function; applies labels to data
tkn_train = train_ds.map(tokenize, batched=True, remove_columns=train_ds.column_names)
tkn_test = test_ds.map(tokenize, batched=True, remove_columns=test_ds.column_names)

Map: 100%|██████████| 57/57 [00:00<00:00, 8409.85 examples/s]


In [7]:
from transformers import Trainer, TrainingArguments
args = TrainingArguments(
    output_dir="exported/gpt2_model/training_arguments",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="steps",
    eval_steps=5_000,
    logging_steps=5_000,
    gradient_accumulation_steps=8,
    num_train_epochs=100,
    weight_decay=0.1,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=5_000,
    # fp16=False, # set to true if using GPU; restart kernel to take effect
    push_to_hub=False,
    )

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tkn_train,
    eval_dataset=tkn_test,
)

# execute the trainer
trainer.train()

/Users/saideeptiku/.miniconda3/envs/hf/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
100%|██████████| 100/100 [00:17<00:00,  5.63it/s]

{'train_runtime': 17.7711, 'train_samples_per_second': 67.525, 'train_steps_per_second': 5.627, 'train_loss': 0.2638910865783691, 'epoch': 100.0}


TrainOutput(global_step=100, training_loss=0.2638910865783691, metrics={'train_runtime': 17.7711, 'train_samples_per_second': 67.525, 'train_steps_per_second': 5.627, 'total_flos': 19596902400000.0, 'train_loss': 0.2638910865783691, 'epoch': 100.0})

In [8]:
# save the model
model.save_pretrained("exported/gpt2_model/model")

In [9]:
import torch
from transformers import pipeline

# which device to use
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# load model and tokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer

infer_model = AutoModelForCausalLM.from_pretrained("exported/gpt2_model/model")
infer_tokenizer = AutoTokenizer.from_pretrained("exported/gpt2_model/tokenizer/")

pipe = pipeline(
    "text-generation", # mode of operation
    model=infer_model,
    tokenizer=infer_tokenizer,
    device=device,
    
    )

In [11]:
print(pipe("from", num_return_sequences=1)[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


from push_to_hub(self, repo_id: str, use_temp_dir: Optional[bool] = None, commit_message_message[bool] = None, str, safe_ --------------------------------: str = None,


In [27]:
# infer_tokenizer.decode(
output = infer_model.generate(
        **infer_tokenizer(["hello"],return_tensors="pt"),
        max_new_tokens=1, 
        # return_dict_in_generate=True, output_scores=True
    )
# )
output[0]
infer_tokenizer.decode(output[0])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


'hello_'